In [1]:
import nltk
nltk.download('punkt')       
nltk.download('stopwords') 
nltk.download('punkt_tab')  
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import numpy as np
import pandas as pd
from helper_functions import *
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rebekaheichberg/nltk_data...

In [2]:
# Load in Recall data

# Paths to all three files
recall_files = [
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_ArtsandCrafts.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Riding_Toys.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Toys.csv"
]

recall_dfs = [load_clean_csv(path) for path in recall_files]
recalls_df = pd.concat(recall_dfs, ignore_index=True)



In [3]:
# Load in reviews data
reviews_df = pd.read_pickle('reviews_raw.pkl')
reviews_df['asin'].nunique()

624792

In [4]:
# initialize model to create embeddings on incident description text
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
# from the recalls data, embed the incident description
combined_indicent_text = " ".join(recalls_df['Incident Description'].dropna().tolist())
incident_desc_embedding = model.encode(combined_indicent_text)
incident_desc_embedding = np.array(incident_desc_embedding).reshape(1,-1)


In [ ]:
#drop na
reviews_model_df = reviews_df[['asin', 'reviewText', 'summary' ,'overall']].copy()
reviews_model_df = reviews_model_df.dropna(subset=['asin','reviewText', 'summary'])

In [8]:
# TODO: should we be dropping duplicates?
reviews_model_df[reviews_model_df.duplicated(['asin', 'reviewText', 'summary'])]

,asin,reviewText,summary,overall
811,0786950072,great game,Five Stars,5
1037,0786955708,Awesome,Five Stars,5
1145,0786955554,nice,Five Stars,5
1155,0786955554,On-time and nice item.,Five Stars,5
1414,0786964502,Great expansion to a great game.,Five Stars,5
...,...,...,...,...
8194060,B01H7VM8P4,As expected,Five Stars,5
8197349,B01HE18R1Q,works great,Five Stars,5
8199066,B01HGKDE22,very pleased with promptness and quality,Five Stars,5
8199586,B01HH7TP68,Super cute,Five Stars,5


In [9]:
# strip possible leading or trailing white space
reviews_model_df = reviews_model_df[reviews_model_df['reviewText'].str.strip() != '']


In [ ]:
# embed the reviewsText and summary columns line by line
# use batching
#reviews_model_df['reviews_embedding'] = reviews_model_df['reviewText'].apply(lambda x: model.encode(x, show_progress_bar=False))

reviews_embeddings = model.encode(
    reviews_model_df['reviewText'].tolist(),
    batch_size=32,    #32, 64, 128 based on memory           
    show_progress_bar=True,
    convert_to_numpy=True        
)


Batches:   0%|          | 0/255994 [00:00<?, ?it/s]

In [ ]:
reviews_embeddings = reviews_model_df['reviews_embedding']
reviews_embeddings = np.array(reviews_embeddings)
reviews_embeddings = np.vstack(reviews_embeddings)

In [ ]:
# take cosine similarity between reviews and the incident description embedding
review_similarities = cosine_similarity(incident_desc_embedding, reviews_embeddings)[0]

In [ ]:
# put the cosine similarities between review and incidents on the main df
reviews_model_df['review_cosine_sim'] = review_similarities

In [ ]:
reviews_model_df

,asin,reviewText,summary,overall,reviews_embedding,summary_embedding,review_cosine_sim
5646013,B01F8QAW7G,who needs a big one xD this little one give me...,tons of small fun to have,5,"[0.055888034, 0.046220485, 0.01708811, -0.0180...","[0.08261334, 0.050293367, 0.09540213, 0.039149...",0.044457
1294965,B001RQDSJQ,If you want a simple gift that has a powerful ...,Girlfriend loves it!,5,"[-0.09749301, 0.035419546, 0.06202652, 0.00467...","[0.0006018644, 0.017727891, 0.043425392, -0.06...",0.197620
2886523,B00C81LYH2,Every year these dolls are purchased for relat...,Beautiful,5,"[-0.035739776, -0.013009322, 0.049390927, -0.0...","[-0.048665345, 0.042406563, 0.052611336, -0.04...",0.176082
1606141,B003OXQ02M,Snap tight model that doesn't snap tight. Best...,"If you want to be frustrated, this is the mode...",1,"[-0.042108443, 0.020145288, -0.0050535235, -0....","[0.006039438, -0.030722562, 0.020900292, 0.027...",0.069005
4793634,B00YHQ74PC,Great!,Five Stars,5,"[-0.02189211, -0.07117047, -0.08756673, -0.027...","[-0.086447485, -0.009291393, 0.025128433, 0.07...",-0.028238
...,...,...,...,...,...,...,...
5970946,B000UJXC1K,I ordered this before reading the reviews and ...,My daughters new favorite toy.,5,"[-0.0030840433, 0.06962656, 0.059129596, 0.025...","[-0.057026964, -0.042803645, 0.08047329, -0.01...",0.343067
7269778,B00JNL5P70,Excellent,Five Stars,5,"[-0.11052086, 0.03127173, -0.04742272, 0.04114...","[-0.086447485, -0.009291393, 0.025128433, 0.07...",-0.012486
5521526,B01CSEHZQ2,Awesome daughter loves it,Five Stars,5,"[-0.027491488, -0.011459269, 0.05401552, -0.04...","[-0.086447485, -0.009291393, 0.025128433, 0.07...",0.219863
3466989,B00HIWWIOE,Like the size of this pool is not too big and ...,Great product!,5,"[0.0139770275, 0.058314063, 0.035566133, -0.02...","[-0.07350235, 0.09336075, -0.00047170976, -0.0...",0.041238


In [ ]:
aggregation_df = reviews_model_df.groupby('asin')['review_cosine_sim'].agg(
    mean_similarity = 'mean',
    max_similarity = 'max'
).reset_index()

In [ ]:
from tqdm import tqdm
# classify the summary in the reviews data
batch_size = 8
texts = reviews_model_df['summary'].fillna('').tolist()
original_indices = reviews_model_df.index.tolist()

scored_rows = []
for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    batch_indices = original_indices[i:i+batch_size]
    
    scores = classify_batch_all_scores(batch)
    
    for idx, score_dict in zip(batch_indices, scores):
        scored_rows.append((idx, score_dict))

label_scores_df = pd.DataFrame(
    [score_dict for idx, score_dict in scored_rows],
    index=[idx for idx, score_dict in scored_rows]
)

reviews_model_df = reviews_model_df.join(label_scores_df)







  0%|          | 0/94 [00:00<?, ?it/s]The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use mps:0
  1%|          | 1/94 [00:01<01:43,  1.11s/it]The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device`

In [ ]:
reviews_model_df

,asin,reviewText,summary,overall,reviews_embedding,summary_embedding,review_cosine_sim,hazardous,choke,harmful,caused injury,accident,dangerous,toxic
5646013,B01F8QAW7G,who needs a big one xD this little one give me...,tons of small fun to have,5,"[0.055888034, 0.046220485, 0.01708811, -0.0180...","[0.08261334, 0.050293367, 0.09540213, 0.039149...",0.044457,0.246170,0.187997,0.183887,0.145903,0.087844,0.085332,0.062867
1294965,B001RQDSJQ,If you want a simple gift that has a powerful ...,Girlfriend loves it!,5,"[-0.09749301, 0.035419546, 0.06202652, 0.00467...","[0.0006018644, 0.017727891, 0.043425392, -0.06...",0.197620,0.114359,0.203966,0.152818,0.103906,0.094309,0.255441,0.075201
2886523,B00C81LYH2,Every year these dolls are purchased for relat...,Beautiful,5,"[-0.035739776, -0.013009322, 0.049390927, -0.0...","[-0.048665345, 0.042406563, 0.052611336, -0.04...",0.176082,0.124837,0.321814,0.112642,0.106995,0.129035,0.088031,0.116646
1606141,B003OXQ02M,Snap tight model that doesn't snap tight. Best...,"If you want to be frustrated, this is the mode...",1,"[-0.042108443, 0.020145288, -0.0050535235, -0....","[0.006039438, -0.030722562, 0.020900292, 0.027...",0.069005,0.184945,0.194804,0.222982,0.087345,0.064118,0.185071,0.060736
4793634,B00YHQ74PC,Great!,Five Stars,5,"[-0.02189211, -0.07117047, -0.08756673, -0.027...","[-0.086447485, -0.009291393, 0.025128433, 0.07...",-0.028238,0.226895,0.103809,0.183021,0.132996,0.068219,0.172712,0.112347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970946,B000UJXC1K,I ordered this before reading the reviews and ...,My daughters new favorite toy.,5,"[-0.0030840433, 0.06962656, 0.059129596, 0.025...","[-0.057026964, -0.042803645, 0.08047329, -0.01...",0.343067,0.106004,0.353527,0.104044,0.093239,0.107165,0.126441,0.109580
7269778,B00JNL5P70,Excellent,Five Stars,5,"[-0.11052086, 0.03127173, -0.04742272, 0.04114...","[-0.086447485, -0.009291393, 0.025128433, 0.07...",-0.012486,0.226895,0.103809,0.183021,0.132996,0.068219,0.172712,0.112347
5521526,B01CSEHZQ2,Awesome daughter loves it,Five Stars,5,"[-0.027491488, -0.011459269, 0.05401552, -0.04...","[-0.086447485, -0.009291393, 0.025128433, 0.07...",0.219863,0.226895,0.103809,0.183021,0.132996,0.068219,0.172712,0.112347
3466989,B00HIWWIOE,Like the size of this pool is not too big and ...,Great product!,5,"[0.0139770275, 0.058314063, 0.035566133, -0.02...","[-0.07350235, 0.09336075, -0.00047170976, -0.0...",0.041238,0.131212,0.211696,0.136250,0.131922,0.133694,0.130140,0.125086
